In [1]:
import os

In [2]:
%pwd

'd:\\Project\\Chest Cancer Classification\\End-to-End-Chest-Cancer-Classification-using-MLflow-and-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%ls

 Volume in drive D is New Volume
 Volume Serial Number is CA1A-9511

 Directory of d:\Project\Chest Cancer Classification\End-to-End-Chest-Cancer-Classification-using-MLflow-and-DVC

07-07-2024  03:03    <DIR>          .
02-07-2024  02:21    <DIR>          ..
01-07-2024  03:43    <DIR>          .github
03-07-2024  01:42             3,251 .gitignore
07-07-2024  03:03    <DIR>          artifacts
02-07-2024  19:40    <DIR>          config
01-07-2024  03:43                 0 dvc.yaml
01-07-2024  02:46             1,088 LICENSE
02-07-2024  00:54    <DIR>          logs
07-07-2024  02:48             1,762 main.py
07-07-2024  02:50               146 params.yaml
07-07-2024  03:01             2,922 README.md
02-07-2024  00:02               195 requirements.txt
06-07-2024  04:22    <DIR>          research
07-07-2024  04:10                73 scores.json
01-07-2024  23:53               830 setup.py
02-07-2024  00:06    <DIR>          src
02-07-2024  21:21             1,298 template.py
01-07-2024  0

In [5]:
%pwd

'd:\\Project\\Chest Cancer Classification\\End-to-End-Chest-Cancer-Classification-using-MLflow-and-DVC'

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/keyurdD1010/End-to-End-Chest-Cancer-Classification-using-MLflow-and-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="keyurdD1010"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "35e6c3ec1e2b0ced9638d403c4bad9ffe7b189be"

In [7]:
os.environ["MLFLOW_TRACKING_URI"]

'https://dagshub.com/keyurdD1010/End-to-End-Chest-Cancer-Classification-using-MLflow-and-DVC.mlflow'

In [8]:
# pip install dagshub

In [9]:
import tensorflow as tf

In [10]:
model = tf.keras.models.load_model("artifacts\\training\\model.h5")

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [12]:

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [13]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Data",
            mlflow_uri="https://dagshub.com/keyurdD1010/End-to-End-Chest-Cancer-Classification-using-MLflow-and-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [14]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [15]:
# class Evaluation:
#     def __init__(self, config: EvaluationConfig):
#         self.config = config

    
#     def _valid_generator(self):

#         datagenerator_kwargs = dict(
#             rescale = 1./255,
#             validation_split=0.30
#         )

#         dataflow_kwargs = dict(
#             target_size=self.config.params_image_size[:-1],
#             batch_size=self.config.params_batch_size,
#             interpolation="bilinear"
#         )

#         valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
#             **datagenerator_kwargs
#         )

#         self.valid_generator = valid_datagenerator.flow_from_directory(
#             directory=self.config.training_data,
#             subset="validation",
#             shuffle=False,
#             **dataflow_kwargs
#         )


#     @staticmethod
#     def load_model(path: Path) -> tf.keras.Model:
#         return tf.keras.models.load_model(path)
    

#     def evaluation(self):
#         self.model = self.load_model(self.config.path_of_model)
#         self._valid_generator()
#         self.score = self.model.evaluate(self.valid_generator)
#         self.save_score()

#     def save_score(self):
#         scores = {"loss": self.score[0], "accuracy": self.score[1]}
#         save_json(path=Path("scores.json"), data=scores)

   


    
#     def log_into_mlflow(self):
    
#         print(self.config.all_params)
#         mlflow.set_registry_uri(self.config.mlflow_uri)
#         tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme        
#         with mlflow.start_run():
#             mlflow.log_params({'AUGMENTATION': str(True),'IMAGE_SIZE': str([224, 224, 3]),'BATCH_SIZE': str(16),'INCLUDE_TOP': str(False),'EPOCHS': str(1),'CLASSES': str(3),'WEIGHTS': 'imagenet','LEARNING_RATE': str(0.01)})
#             mlflow.log_metrics(
#                 {"loss": self.score[0], "accuracy": self.score[1]}
#             )
#             # Model registry does not work with file store
#             if tracking_url_type_store != "file":

#                 # Register the model
#                 # There are other ways to use the Model Registry, which depends on the use case,
#                 # please refer to the doc for more information:
#                 # https://mlflow.org/docs/latest/model-registry.html#api-workflow
#                 mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
#             else:
#                 mlflow.keras.log_model(self.model, "model")


import mlflow
from urllib.parse import urlparse
import tensorflow as tf
from pathlib import Path

class Evaluation:
    def __init__(self, config):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        
        param = self.config.all_params
        matric = {"loss": self.score[0], "accuracy": self.score[1]}
        # print("paramsss",param)
        # print("matrics",matric)
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params( {
    "AUGMENTATION": True,
    "IMAGE_SIZE": [224, 224, 3],    
    "BATCH_SIZE": 16,             
    "INCLUDE_TOP": False,           
    "EPOCHS": 1,            
    "CLASSES": 3,                
    "WEIGHTS": 'imagenet',     
    "LEARNING_RATE": 0.01          
}
)
            mlflow.log_metrics(matric)
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [16]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-07-07 14:26:26,174: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-07 14:26:26,258: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-07 14:26:26,258: INFO: common: created directory at: artifacts]
Found 298 images belonging to 3 classes.
19/19 [==============================] - 74s 4s/step - loss: 19.1766 - accuracy: 0.3154
[2024-07-07 14:27:43,568: INFO: common: json file saved at: scores.json]


RestException: INVALID_PARAMETER_VALUE: Response: {'error_code': 'INVALID_PARAMETER_VALUE'}

In [ ]:
try:
    # config = ConfigurationManager()
    # eval_config = config.get_evaluation_config()
    # evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    # evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 298 images belonging to 3 classes.
19/19 [==============================] - 75s 4s/step - loss: 19.1766 - accuracy: 0.3154
[2024-07-07 03:13:44,339: INFO: common: json file saved at: scores.json]


In [19]:
try:
    # config = ConfigurationManager()
    # eval_config = config.get_evaluation_config()
    # evaluation = Evaluation(eval_config)
    # evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

{'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 16, 'INCLUDE_TOP': False, 'EPOCHS': 1, 'CLASSES': 3, 'WEIGHTS': 'imagenet', 'LEARNING_RATE': 0.01}


RestException: INVALID_PARAMETER_VALUE: Response: {'error_code': 'INVALID_PARAMETER_VALUE'}

In [11]:
import mlflow

# Set the MLflow tracking URI if you have a remote server
mlflow.set_tracking_uri("https://dagshub.com/keyurdD1010/End-to-End-Chest-Cancer-Classification-using-MLflow-and-DVC.mlflow")

# Set or create an experiment
experiment_name = "demo1"
mlflow.set_experiment(experiment_name)

# Define the parameters for the VGG16 model
model_params = {
    'AUGMENTATION': str(True),
    'IMAGE_SIZE': str([224, 224, 3]),  # Convert list to string
    'BATCH_SIZE': str(16),
    'INCLUDE_TOP': str(False),
    'EPOCHS': str(1),
    'CLASSES': str(3),
    'WEIGHTS': 'imagenet',            # Already a string
    'LEARNING_RATE': str(0.01)
}

# Start an MLflow run
with mlflow.start_run():
    mlflow.log_params(model_params)


2024/07/07 03:23:58 INFO mlflow.tracking.fluent: Experiment with name 'demo1' does not exist. Creating a new experiment.
